This notebook includes all of the code for the experients and analyses.  It uses a python v3 and tensorflow v1.14 port of the SEM model, and should run as is in Colab.  

I've hidden most of the code except for the relevant parameters to switch between Blocked > Interleaved and Interleaved > Blocked.  Hopefully the plots are clear, but let me know if they are not.



## Load Libraries

In [17]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
from sklearn import metrics
import pandas as pd

from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import euclidean_distances
from scipy.special import logsumexp
from scipy.stats import norm
from glob import glob

#this code just allows us to add the custom code 
# import sys, os
# current_path = os.path.abspath('.')
# parent_path = os.path.dirname(current_path)
# sys.path.append(parent_path)

from schema_prediction_task_9_8_20 import generate_exp, batch_exp
%matplotlib inline

In [18]:
from vanilla_lstm import VanillaLSTM
from sem.event_models import NonLinearEvent

# Run model

The main script we use to run the experiment is  `schema_prediction_task_9_8_20.batch_exp`.  It's a wrapper function for a lot of code,
but it will generate a set of randomly sampled tasks of the specified conditions (default = Blocked, Interleaved), run the model (either SEM or a "no_split" variant of SEM that collapses to the NN specified) and calculate the meaningful metrics of the task.


It's desined to be a parallelizable job on a cluster -- you can pass a set of parameters to the function and get a pair of files as an output that represents a random sample of behavior for those parameters on the specified tasks.  I (NTF) would spawn thousands of instances of this function in seperate SLURM jobs, each with different parameters and write the results to file for grid-searches.  It is possible to get fancier with the paralization by writting a custom wrapper for some of the SEM module (look at the function `no_split_sem.no_split_sem_run_with_boundaries` for ideas) but I chose not to.

### Parameters

In [19]:
seed = None
err = 0.2; # error probability for plate's formula

In [26]:
## sem parameters
lr                = 0.05
##
dropout           = 0.0  # don't change
l2_regularization = 0.0  # don't change
epsilon           = 1e-5 # don't change
log_alpha         = 0.0  # sCRP alpha is set in log scale
log_lambda        = 0.0   # sCRP lambda is set in log scale
n_hidden          = None # don't change -- set the number of hidden units equal to the dimensionality
##
batch_size        = 25   # don't change
batch_update      = False # batch update (True) or local gradient updating (False)
#
exp_name = 'EXP1'


In [4]:
# ADAM parameters, don't change
optimizer_kwargs = dict(
    lr=lr, beta_1=0.9, beta_2=0.999, epsilon=epsilon, amsgrad=False
)

# set the event model class
f_class = VanillaLSTM

### dicts setup

In [5]:
n_epochs = 2 # not sure what n_epochs does

f_opts=dict(
  n_epochs = n_epochs, 
  batch_size = batch_size,
  batch_update = batch_update,
  l2_regularization = l2_regularization,  
  dropout = dropout,
  optimizer_kwargs = optimizer_kwargs
)

sem_kwargs = dict(
  lmda=np.exp(log_lambda), 
  alfa=np.exp(log_alpha), 
  f_opts=f_opts, 
  f_class=f_class)

### run SEM

In [16]:
n_batch = 2 # number of sims per model

# story_generator = generate_exp_balanced
story_kwargs = dict(seed=seed, err=err, actor_weight=1.0, instructions_weight=1.0)
x, y, e, embedding_library = generate_exp('blocked', **story_kwargs)


results, trialXtrial, _ = batch_exp(
  sem_kwargs, story_kwargs, 
  n_batch=n_batch, sem_progress_bar=True, progress_bar=False, 
  block_only=False, run_instructed=True, 
  no_split=True # toggle between SEM (False) and LSTM (True) 
)


Run SEM:  38%|███▊      | 76/200 [00:33<00:54,  2.30it/s]
Process Process-9:
Traceback (most recent call last):
  File "//anaconda3/envs/sem/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "//anaconda3/envs/sem/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/abeukers/wd/csw/SchemaPrediction_internal/no_split_sem.py", line 365, in worker_run_with_boundaries
    sem_model.run_w_boundaries(x, **run_kwargs)
  File "/Users/abeukers/wd/csw/SchemaPrediction_internal/no_split_sem.py", line 335, in run_w_boundaries
    self.update_single_event(x, save_x_hat=save_x_hat)
  File "/Users/abeukers/wd/csw/SchemaPrediction_internal/no_split_sem.py", line 257, in update_single_event
    self.event_models[k].update(x_prev, X0)
  File "//anaconda3/envs/sem/lib/python3.7/site-packages/sem/event_models.py", line 592, in update
    self.estimate()
  File "//anaconda3/envs/sem/lib/python3.7/site-p

KeyboardInterrupt: 

  File "//anaconda3/envs/sem/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 4084, in __init__
    use_legacy_function=use_legacy_function)
  File "//anaconda3/envs/sem/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 3371, in __init__
    self._function = wrapper_fn.get_concrete_function()
  File "//anaconda3/envs/sem/lib/python3.7/site-packages/tensorflow/python/eager/function.py", line 2939, in get_concrete_function
    *args, **kwargs)
  File "//anaconda3/envs/sem/lib/python3.7/site-packages/tensorflow/python/eager/function.py", line 2906, in _get_concrete_function_garbage_collected
    graph_function, args, kwargs = self._maybe_define_function(args, kwargs)
  File "//anaconda3/envs/sem/lib/python3.7/site-packages/tensorflow/python/eager/function.py", line 3213, in _maybe_define_function
    graph_function = self._create_graph_function(args, kwargs)
  File "//anaconda3/envs/sem/lib/python3.7/site-packages/tensorflow/python/ea

In [13]:
# convert results files from JSON (dict) to pandas for saving
results = pd.DataFrame(results)
trialXtrial = pd.DataFrame(trialXtrial)

### save to csv

In [27]:
model_ftag = '_{}_nhidden{}_e{}_lr{}_n{}_d{}_logalfa_{}_loglmda_{}'.format(
        exp_name, n_hidden, epsilon, lr, n_epochs, dropout, log_alpha, log_lambda)
model_ftag

'_EXP1_nhiddenNone_e1e-05_lr0.05_n2_d0.0_logalfa_0.0_loglmda_0.0'

In [28]:
trialXtrial

,t,e_hat,Accuracy,pe,batch,Condition
0,0,0,0.497093,0.835935,0,Blocked
1,1,0,0.525540,0.556829,0,Blocked
2,2,0,0.642519,0.503048,0,Blocked
3,3,0,0.471595,0.671526,0,Blocked
4,4,0,0.513478,0.702721,0,Blocked
...,...,...,...,...,...,...
795,195,0,0.752081,0.520768,1,Interleaved
796,196,0,0.538967,0.648266,1,Interleaved
797,197,0,0.555756,0.627615,1,Interleaved
798,198,0,0.206139,0.583621,1,Interleaved


In [15]:
results

,Trials,adjRand,nClusters,pe,pe (probes),verb decoder Accuracy,verb decoder Accuracy Prob,verb 2 AFC decoder Prob,batch,Condition,cluster re-use
0,All,0.0,1,0.577114,0.584239,0.167500,0.152348,0.534867,0,Blocked,NaN
1,Training,0.0,1,0.575171,0.581781,0.165625,0.149332,0.530845,0,Blocked,NaN
2,Test,0.0,1,0.584885,0.594070,0.175000,0.164411,0.550953,0,Blocked,1.0
3,All,0.0,1,0.563320,0.554572,0.190000,0.183066,0.520043,0,Interleaved,NaN
4,Training,0.0,1,0.566292,0.558593,0.184375,0.184390,0.516440,0,Interleaved,NaN
5,Test,0.0,1,0.551430,0.538488,0.212500,0.177769,0.534452,0,Interleaved,1.0
6,All,0.0,1,0.577409,0.581429,0.110000,0.125789,0.524196,1,Blocked,NaN
7,Training,0.0,1,0.570140,0.572768,0.112500,0.133198,0.536254,1,Blocked,NaN
8,Test,0.0,1,0.606486,0.616075,0.100000,0.096153,0.475965,1,Blocked,1.0
9,All,0.0,1,0.572069,0.600138,0.095000,0.099365,0.464746,1,Interleaved,NaN
